# Just want to play with the OOI API 
I want to basically follow some of the simple examples from the OOI workshops to get and plot data


In [11]:
USERNAME  = 'OOIAPI-M3ZHX5W9PVAPNQ'
TOKEN =  'TEMP-TOKEN-MM09XIOFPM8QV2'

# the sensor base URL is the path to all sensor info
SENSOR_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

In [2]:
# First, we need to add some more Python libraries
import requests
import datetime

In [3]:
# get a list of all the site info:
r = requests.get(SENSOR_BASE_URL, auth=(USERNAME, TOKEN))

# now output all the site info:

sites = r.json()
for site in sites:
  print( site )

CE01ISSM
CE01ISSP
CE02SHBP
CE02SHSM
CE02SHSP
CE04OSBP
CE04OSPD
CE04OSPI
CE04OSPS
CE04OSSM
CE05MOAS
CE06ISSM
CE06ISSP
CE07SHSM
CE07SHSP
CE09OSPM
CE09OSSM
CP01CNPM
CP01CNSM
CP01CNSP
CP02PMCI
CP02PMCO
CP02PMUI
CP02PMUO
CP03ISPM
CP03ISSM
CP03ISSP
CP04OSPM
CP04OSSM
CP05MOAS
GA01SUMO
GA02HYPM
GA03FLMA
GA03FLMB
GA05MOAS
GI01SUMO
GI02HYPM
GI03FLMA
GI03FLMB
GI05MOAS
GP02HYPM
GP03FLMA
GP03FLMB
GP05MOAS
GS01SUMO
GS02HYPM
GS03FLMA
GS03FLMB
GS05MOAS
RS01OSBP
RS01SBPD
RS01SBPS
RS01SHBP
RS01SHDR
RS01SLBS
RS01SUM1
RS01SUM2
RS03ASHS
RS03AXBS
RS03AXPD
RS03AXPS
RS03AXSM
RS03CCAL
RS03ECAL
RS03INT1
RS03INT2
SSRSPACC


In [4]:
# A quick function to make an API request and print the results
def get_and_print_api(url):
  r = requests.get(url, auth=(USERNAME, TOKEN))
  data = r.json()
  for d in data:
    print( d )

List of nodes for a site

In [5]:
# lets look at the Irminger sea site
get_and_print_api(SENSOR_BASE_URL+'/GI02HYPM')

RIM01
WFP02


In [6]:
# lets look at the profiler instruments
get_and_print_api(SENSOR_BASE_URL+'/GI02HYPM/WFP02')

00-WFPENG000
01-FLORDL000
03-DOSTAL000
04-CTDPFL000
05-VEL3DL000


In [7]:
# list the methods for the sensor
get_and_print_api(SENSOR_BASE_URL+'/GI02HYPM/WFP02/04-CTDPFL000')

recovered_wfp
telemetered


In [10]:
# list the streams for a method
get_and_print_api(SENSOR_BASE_URL+'/GI02HYPM/WFP02/04-CTDPFL000/recovered_wfp')

ctdpf_ckl_wfp_instrument_recovered
ctdpf_ckl_wfp_metadata_recovered



From here if we add one of these streams to the data request, we will get the data. This is an asynchronous request, and we will get all of it. Sounds like this causes some problems on the server side right now

Instead, we should set up a synchronous request that sends back a quick decimated version of the data for Expoloator purposes

# Create quick, EDA, synchronous request